<a href="https://colab.research.google.com/github/Axle-Bucamp/Recomandation-System-Gan/blob/main/recomandation_system_CalculDiversityOverTime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import date
import datetime
import numpy as np
import h5py
import tensorflow as tf
from tensorflow import keras
from ipywidgets import widgets
import random

user = pd.read_csv('ratings.csv')
#m_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
#movie_name = pd.read_csv('u.item', sep='|', names=m_cols, usecols=range(5), encoding="iso-8859-1")
#id_select = user.copy()
user = user.iloc[0:200000]
user = user[user["movieId"] < 13816]
user.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [ ]:
user["timestamp"] = (user["timestamp"] - user["timestamp"].min()) 
user["timestamp"] = user["timestamp"] / user["timestamp"].max()

In [ ]:
table = pd.pivot_table(user, values='rating', index=['userId'],columns=['movieId'], aggfunc=np.sum)
table_timestamp = pd.pivot_table(user, values='timestamp', index=['userId'],columns=['movieId'], aggfunc=np.sum)

ZerosCol = table.isna().sum(axis = 0)
ZerosRow = table.isna().sum(axis = 1)

table = table.fillna(0)
table_timestamp = table_timestamp.fillna(0)

In [ ]:
table.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,8994,8995,8997,8998,8999,9001,9004,9005,9010,9018
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
table[1]

userId
1       0.0
2       3.5
3       4.0
4       3.0
5       4.0
       ... 
1405    0.0
1406    0.0
1407    0.0
1408    0.0
1409    4.0
Name: 1, Length: 1406, dtype: float64

In [ ]:
np.array(table)[:,0]

array([0. , 3.5, 4. , ..., 0. , 0. , 4. ])

In [ ]:
np.unique(user["movieId"]).shape

(6163,)

In [ ]:
model = tf.keras.models.load_model("Generator.h5", compile=False)

In [ ]:
UserRow = 128

In [ ]:
genome_tag = pd.read_csv('../ml-25m/genome-scores.csv')
genome_matrix = pd.pivot_table(genome_tag, values='relevance', index=['movieId'],columns=['tagId'], aggfunc=np.sum)

In [ ]:
from scipy.spatial.distance import pdist
def rel_proximity(movieList):
    return np.mean(pdist(genome_matrix.iloc[movieList] , 'euclidean'))

In [ ]:
def create_input_matrix(choice,value):
  number_of_rows = table.shape[0]
  movielist = list(np.unique(user["movieId"]))

  final = choice.copy()
  for x in choice :
    if x in movielist:
      movielist.remove(x)

  movielist = np.array(movielist)
  random_col = np.random.choice(movielist.shape[0] , size=128 - len(choice), replace=False)
  random_col = [*movielist[random_col], *choice]

  Xpred = table.copy()    
  Xpred[:].iloc[0] = 0
  Xpred[final].iloc[0] = value
  
  ligne_user = np.array(Xpred[random_col].iloc[0]).copy()

  Xpred = Xpred[random_col]
  Xpred = np.array(Xpred)

  row = []
  for i in range(number_of_rows) :
    if np.sum(Xpred[i,:] > 0 ) > 10:
      row.append(i)

  while len(row) < 128:
    row.append(random.randint(0, 943))
    
  random_row = np.random.choice(row, size=128, replace=False)
  Xpred = Xpred[random_row,:]
  Xpred[0,:] = ligne_user
  #mean_matrix = np.ones((128,128)) * mean[random_col] 

  Xpred = Xpred.reshape(128,128,1)
  time_table = table_timestamp[random_col]
  time_table = time_table.iloc[random_row]
  time_table = np.array(time_table)
  time_table = time_table.reshape(128,128,1)

  Xpred = np.concatenate([Xpred, time_table], axis=2)
  Xpred = np.concatenate([Xpred, np.random.normal(0,1,128*128).reshape(128,128,1) * 0.1 ], axis=2) # mean_matrix.reshape(128,128,1) +

  return Xpred, random_col


In [ ]:
def prediction_call(model, choice, value):
  xpred, rand_col = create_input_matrix(choice,value)
  ypred = model(tf.convert_to_tensor(xpred.reshape(1,128,128,3).astype(float))).numpy().reshape(128,128)
  max_indice = np.argpartition(ypred[0,:], -10)[-10:]
  return rand_col[max_indice[0]], random.randint(3, 5),rel_proximity(np.array(rand_col)[max_indice])


In [ ]:
import random
pd.options.mode.chained_assignment = None  # default='warn'

# recommandé
distance_intra_reco = []
distance_reco = []
for y in range(100):
    choice = [298]
    value = [4]
    distance = []
    print(y)
    for i in range(10):
        new_choice,new_value,dist_intra = prediction_call(model,choice,value)
        distance_intra_reco.append(dist_intra)
        choice.append(new_choice)
        value.append(new_value)
        dist = rel_proximity(choice)
        distance.append(dist)
    distance_reco.append(distance)
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [ ]:
#non recommandé
movieList = np.unique(user["movieId"])
distance_noreco = []
for y in range(100):
    choice = [298]
    value = [4]
    distance = []
    print(y)
    for i in range(10):
        new_choice = random.choice(list(np.unique(user["movieId"])))
        new_value = random.randint(3, 5)
        choice.append(new_choice)
        value.append(new_value)
        dist = rel_proximity(choice)
        distance.append(dist)
    distance_noreco.append(distance)
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [ ]:
distance_intra_alea = []
for y in range(100):
    for i in range(10):
        new_choice = []
        for z in range(10):
            new_choice.append(random.choice(list(np.unique(user["movieId"]))))
        distance_intra_alea.append(rel_proximity(new_choice))


In [ ]:
from scipy.stats import ttest_ind
distance_reco = np.array(distance_reco)
distance_noreco = np.array(distance_noreco)
for i in range(10) :
    print("number of movie seen {} :".format(i))
    print(ttest_ind(distance_reco[:,i], distance_noreco[:,i]))


number of movie seen 0 :
Ttest_indResult(statistic=-0.7481602144943595, pvalue=0.4552512481013098)
number of movie seen 1 :
Ttest_indResult(statistic=-1.4679668950443818, pvalue=0.1436999751251407)
number of movie seen 2 :
Ttest_indResult(statistic=-1.9129327813902832, pvalue=0.057199568903675065)
number of movie seen 3 :
Ttest_indResult(statistic=-2.2292837590750807, pvalue=0.026918850582534168)
number of movie seen 4 :
Ttest_indResult(statistic=-3.179636363615693, pvalue=0.0017115997488416352)
number of movie seen 5 :
Ttest_indResult(statistic=-2.8046141069947006, pvalue=0.0055397148960124485)
number of movie seen 6 :
Ttest_indResult(statistic=-2.9460542017139644, pvalue=0.0036044735145627864)
number of movie seen 7 :
Ttest_indResult(statistic=-2.936862693160807, pvalue=0.003708369126361719)
number of movie seen 8 :
Ttest_indResult(statistic=-2.985372028470412, pvalue=0.003189478841700707)
number of movie seen 9 :
Ttest_indResult(statistic=-2.952492603640769, pvalue=0.003533293568933

In [ ]:
print("test répartition des distances intra groupes de recommandations")
print(ttest_ind(distance_intra_alea, distance_intra_reco))


test répartition des distances intra groupes de recommandations
Ttest_indResult(statistic=3.3575751383518813, pvalue=0.0008010429604573216)
